In [2]:
#	Pneumonia detection from X-rays	CNN + preprocessing	Chest X-ray dataset
#🔥 Bonus: Use Grad-CAM or saliency maps for interpretability.
import os
print(os.listdir()) 

['best_resnet_pneumonia.pth', 'chestXray', 'pnemonia_prediction.ipynb', 'resnet_pneumonia.pth', 'runs']


In [3]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,Subset
import torchvision
from torchvision import transforms,datasets,models
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from collections import Counter

In [4]:
datadir='chestXray/chest_xray/train'

In [5]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.Grayscale(num_output_channels=3),  # Convert to 3-channel
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
full_dataset_for_split = datasets.ImageFolder(root=datadir, transform=val_test_transform)


In [6]:
train_idx, val_idx = train_test_split(
    list(range(len(full_dataset_for_split))),
    test_size=0.2,
    stratify=[label for _, label in full_dataset_for_split],  # stratify to maintain class balance
    random_state=42
)

train_dataset = Subset(
    datasets.ImageFolder(root=datadir, transform=train_transform),
    train_idx
)
val_dataset = Subset(full_dataset_for_split, val_idx)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

test_dataset=datasets.ImageFolder('chestXray/chest_xray/test',transform=val_test_transform)
test_loader=DataLoader(test_dataset,batch_size=64,shuffle=False)

In [ ]:
# train_transform=transforms.Compose([transforms.RandomResizedCrop(224, scale=(0.8, 1.0), ratio=(0.9, 1.1)),
#                                     transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.9, 1.1)),
#                                     transforms.RandomHorizontalFlip(),
#                                     transforms.RandomRotation(15),
#                                     transforms.ToTensor(),
#                                     transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
#                                     ])
# test_transform=transforms.Compose([transforms.Resize((224,224)),
#                                    transforms.Grayscale(num_output_channels=3),
#                                     transforms.ToTensor(),
#                                     transforms.Normalize([0.485,0.456,0.406],
#                                                          [0.229,0.224,0.225])
#                                                          ])
# train_dataset=datasets.ImageFolder('chestXray/chest_xray/train',transform=train_transform)
# train_loader=DataLoader(train_dataset,batch_size=64,shuffle=True)

# val_dataset   = datasets.ImageFolder('chestXray/chest_xray/val', transform=test_transform)
# val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [7]:
writer=SummaryWriter("runs/pneumonia_resnet")

#data_dir="chestXray/chest_xray"

In [8]:
# val_labels = [label for _, label in val_dataset]
# Counter(val_labels), len(val_labels)


In [9]:
model=models.resnet18(pretrained=True)
num_features=model.fc.in_features
print(num_features)
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_features, 2)
)



c:\Users\LENOVO\Documents\pytorch\venvtorch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\LENOVO\Documents\pytorch\venvtorch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


512


In [10]:


# Get training labels properly from the Subset
train_labels = [train_dataset.dataset.targets[i] for i in train_dataset.indices]
train_labels_np = np.array(train_labels)

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels_np), y=train_labels_np)
class_weights = torch.tensor(class_weights, dtype=torch.float)

criterion=nn.CrossEntropyLoss(weight=class_weights)
optimizer=optim.Adam(model.parameters(),lr=1e-4,weight_decay=1e-5)


In [11]:
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)
best_val_loss = float('inf')
early_stop_counter = 0
early_stop_patience = 5

In [12]:
no_epochs=30

for epochs in range(no_epochs):
    model.train()
    running_loss=0.0
    correct=0.0
    total=0.0
    loop=tqdm(train_loader,desc=f"Epoch[{epochs+1}]")  
    for images,labels in loop:
        outputs=model(images)
        optimizer.zero_grad()
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()

        running_loss+=loss.item()

        _,predicted=outputs.max(1)
        total+=labels.size(0)
        correct+=(predicted==labels).sum().item()

        loop.set_postfix(loss=loss.item(),accuracy=100*correct/total)
    train_loss = running_loss / len(train_loader)
    train_acc=100*correct/total
    writer.add_scalar("loss/train",running_loss/len(train_loader),epochs)
    writer.add_scalar('accuracy/train',train_acc,epochs)

    model.eval()
    val_loss=0.0
    val_correct=0
    val_total=0

    with torch.no_grad():
        for images,labels in val_loader:
            outputs=model(images)
            loss=criterion(outputs,labels)

            val_loss+=loss.item()
            _,predicted=outputs.max(1)
            val_total+=labels.size(0)
            val_correct+=(predicted==labels).sum().item()
        val_loss /= len(val_loader)
        val_accuracy=100*val_correct/val_total
        writer.add_scalar("loss/val",val_loss/len(val_loader),epochs)
        writer.add_scalar("accuracy/val",val_accuracy,epochs)

        print(f"epoch{epochs+1}:Train Acc:{train_acc}% ,val Acc:{val_accuracy},Train loss:{train_loss},val loss:{val_loss}")
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_resnet_pneumonia.pth')
            early_stop_counter = 0
            print("Validation loss decreased, saving model.")
        else:
            early_stop_counter += 1
            if early_stop_counter >= early_stop_patience:
                print("Early stopping triggered.")
                break





Epoch[1]: 100%|██████████| 66/66 [04:22<00:00,  3.98s/it, accuracy=90.7, loss=0.00778]


epoch1:Train Acc:90.74784276126557% ,val Acc:89.55938697318008,Train loss:0.20278911052666831,val loss:0.2058147099964759
Validation loss decreased, saving model.


Epoch[2]: 100%|██████████| 66/66 [04:31<00:00,  4.11s/it, accuracy=95.8, loss=0.158] 


epoch2:Train Acc:95.80536912751678% ,val Acc:93.5823754789272,Train loss:0.11657510316846045,val loss:0.12979807573206284
Validation loss decreased, saving model.


Epoch[3]: 100%|██████████| 66/66 [03:57<00:00,  3.61s/it, accuracy=96.7, loss=0.04]  


...

epoch9:Train Acc:98.25023969319271% ,val Acc:97.31800766283524,Train loss:0.04728339429041653,val loss:0.06817099117838285


Epoch[10]: 100%|██████████| 66/66 [03:59<00:00,  3.62s/it, accuracy=98.1, loss=0.153]  


epoch10:Train Acc:98.13039309683604% ,val Acc:95.30651340996168,Train loss:0.048219459649937395,val loss:0.09799995539052521


Epoch[11]:  20%|█▉        | 13/66 [01:10<04:49,  5.46s/it, accuracy=98.6, loss=0.139] 


KeyboardInterrupt: 

In [14]:
model.eval() 
correct = 0
total = 0

with torch.no_grad():
    test_loss=0
    for inputs, labels in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

avg_test_loss = test_loss / len(test_loader)
test_accuracy =100*correct / total

print(f"Test Loss: {avg_test_loss}, Test Accuracy: {test_accuracy}")


Test Loss: 0.22756539536640047, Test Accuracy: 93.42948717948718


In [28]:
# import matplotlib.pyplot as plt

# # Show some test images
# for images, labels in test_loader:
#     for i in range(5):
#         img = images[i].permute(1,2,0).numpy()
#         img = img * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406]  # unnormalize
#         plt.imshow(img)
#         plt.title(f"Label: {labels[i]}")
#         plt.show()
#     break


In [27]:
# import matplotlib.pyplot as plt

# # Show some test images
# for images, labels in train_loader:
#     for i in range(5):
#         img = images[i].permute(1,2,0).numpy()
#         img = img * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406]  # unnormalize
#         plt.imshow(img)
#         plt.title(f"Label: {labels[i]}")
#         plt.show()
#     break


In [26]:
from collections import Counter

# Access the labels using indices from the Subset
train_labels = [train_dataset.dataset.targets[i] for i in train_dataset.indices]
test_labels = [test_dataset.targets[i] for i in range(len(test_dataset))]

print("Train class distribution:", Counter(train_labels))
print("Test class distribution:", Counter(test_labels))


Train class distribution: Counter({1: 3099, 0: 1073})
Test class distribution: Counter({1: 390, 0: 234})
